In [ ]:
from flight_fusion import ModelServiceClient
import pyarrow as pa

mc = ModelServiceClient()
mc.list_models().models

In [ ]:
# if the model stare is "UNAVAILABLE", we have to load the model before further usage.
mc.load_model(model_name="ElasticnetWineModel")

In [ ]:
# to interact with the model we create a model client
model = mc.get_model_client(name="ElasticnetWineModel", version="2")
print("Model input schema:\n")
print(model.input_schema, "\n\n")
print("Model output schema:\n")
print(model.output_schema)

In [ ]:
data = [
    pa.array([7.4, 7.4]),
    pa.array([0.7000, 0.7000]),
    pa.array([0, 0]),
    pa.array([1.9, 1.9]),
    pa.array([0.076, 0.076]),
    pa.array([11, 11]),
    pa.array([34, 34]),
    pa.array([0.9978, 0.9978]),
    pa.array([3.51, 3.51]),
    pa.array([0.56, 0.56]),
    pa.array([9.4, 9.4]),
]

table = pa.Table.from_arrays(data, schema=model.input_schema)
model.predict(table=table)

In [ ]:
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split

# The digits dataset
digits = datasets.load_digits()

# To apply a classifier on this data, we need to flatten the image, to
# turn the data in a (samples, feature) matrix:
n_samples = len(digits.images)
data = digits.images.reshape((n_samples, -1))

# Create a classifier: a support vector classifier
classifier = svm.SVC(gamma=0.001)

# Split data into train and test subsets
X_train, X_test, y_train, y_test = train_test_split(
    data, digits.target, test_size=0.5, shuffle=False)

# We learn the digits on the first half of the digits
classifier.fit(X_train, y_train)

In [ ]:
import joblib

model_file_name = "mnist-svm.joblib"
joblib.dump(classifier, model_file_name)

In [ ]:
import os

os.environ["MLFLOW_TRACKING_URI"] = "http://127.0.0.1:5000"

In [ ]:
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature

import logging

logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)


def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

warnings.filterwarnings("ignore")
np.random.seed(40)

mlflow.sklearn.autolog()

# Read the wine-quality csv file from the URL
csv_url = (
    "http://archive.ics.uci.edu/ml"
    "/machine-learning-databases/wine-quality/winequality-red.csv"
)
try:
    data = pd.read_csv(csv_url, sep=";")
except Exception as e:
    logger.exception(
        "Unable to download training & test CSV, "
        "check your internet connection. Error: %s",
        e,
    )

# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(data)

# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]

alpha = 0.5
l1_ratio = 0.5

experiment = mlflow.set_experiment("wine_model_new")
print(experiment)

with mlflow.start_run():
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)

    predicted_qualities = lr.predict(test_x)

    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

    print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)

    # mlflow.log_param("alpha", alpha)
    # mlflow.log_param("l1_ratio", l1_ratio)
    # mlflow.log_metric("rmse", rmse)
    # mlflow.log_metric("r2", r2)
    # mlflow.log_metric("mae", mae)

    tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
    model_signature = infer_signature(train_x, train_y)

    # Model registry does not work with file store
    if tracking_url_type_store != "file":

        # Register the model
        # There are other ways to use the Model Registry,
        # which depends on the use case,
        # please refer to the doc for more information:
        # https://mlflow.org/docs/latest/model-registry.html#api-workflow
        mlflow.sklearn.log_model(
            lr,
            "model",
            registered_model_name="ElasticnetWineModel",
            signature=model_signature,
        )
    else:
        mlflow.sklearn.log_model(lr, "model", signature=model_signature)